# Deploy and run a Kubeflow Pipeline from outside the Kubeflow cluster: non-IAP version

This notebook shows how to deploy and run a [Kubeflow](https://kubeflow.org) Pipeline from outside the Kubeflow cluster.  

In contrast to the [`kfp_remote_deploy-IAP.ipynb`](kfp_remote_deploy-IAP.ipynb) notebook, this example does not require that the cluster has been set up to use Google Cloud Platform's [Identity-Aware Proxy (IAP)](https://cloud.google.com/iap/). 

However, it does assume that you're able to set up a port-forward to the Pipelines UI. For Kubeflow installations that include Pipelines (including the ['lightweight' Kubeflow Pipelines installation](https://github.com/kubeflow/pipelines/tree/master/manifests/kustomize)), you should be able to connect directly to the Pipelines UI as follows, replacing `8080` with the local port you'd like to use.

```sh
kubectl port-forward -n kubeflow svc/ml-pipeline-ui 8080:80
```
For generality of example, we'll use this port-forwarding command for this notebook.

> **Important Note**: the command above will not give you access to the rest of the Kubeflow Dashboard. But, it is a straightforward way to connect in order to deploy pipelines and make other Pipelines API calls.

Alternately, if your Kubeflow cluster has been set up to support port-forwarding, as with the generic [installation instructions here](https://www.kubeflow.org/docs/started/k8s/kfctl-k8s-istio/), you may be able to port-forward via istio to the Kubeflow Dashboard as follows: `kubectl -n istio-system port-forward svc/istio-ingressgateway 8080:80`.

For simplicity of example we do assume a cluster running on [Kubernetes Engine](https://cloud.google.com/kubernetes-engine/) (GKE). However, this basic approach should work for Kubeflow installed on non-GKE  Kubernetes-conformant clusters as well.

## Setup and configuration


### Deploy a Kubeflow cluster on GKE

Deploy a [Kubeflow](https://kubeflow.org) cluster on GKE.  The [launcher web app](https://deploy.kubeflow.cloud/#/deploy) is recommended for ease of use. 
You can also deploy from the command line using the `kfctl` utility, as [described here](https://www.kubeflow.org/docs/gke/deploy/deploy-cli/).

Once the GKE cluster is up and running, visit https://console.cloud.google.com/kubernetes/list and click on the **Connect** button to the right of your Kubeflow cluster. Copy the given command-line access snippet, which should look like:

```
gcloud container clusters get-credentials <your-cluster-name> --zone <your-zone> --project <your-project>
```
You'll need this for the next step.


## Running the notebook example

### Run the example in an AI Platform Notebook


Visit [https://console.cloud.google.com/mlengine/notebooks/instances](https://console.cloud.google.com/mlengine/notebooks/instances) and create a **NEW INSTANCE** (or you can use an existing instance if you prefer).

Once the instance is up and running, click on **OPEN JUPYTERLAB**, and upload this notebook.

Under **File** > **New**, start up a new **Terminal** tab.  In the terminal window, run the `gcloud container clusters get-credentials ...` command described above.
This will auth `kubectl` to connect to your Kubeflow cluster. 

Then, port-forward to the Kubeflow dashboard by running the following command in the terminal window. 

```
kubectl port-forward -n kubeflow svc/ml-pipeline-ui 8089:80
```

(Often the convention is to use port 8080, but in the AI Platform notebook environment that port is already taken, so we're using another, `8089`.)

### Run the example in a local Jupyter installation

To run this notebook locally, you'll need Jupyter and Python 3 installed.  

Then, in your local environment, run the `gcloud container clusters get-credentials ...` command described above.
This will auth `kubectl` to connect to your Kubeflow cluster. 

Then, port-forward to the Kubeflow dashboard by running the `kubectl port-forward ...` command of the previous section.



### The code

Now we're ready to run the example code.

First, install the Kubeflow Pipelines SDK. If you get import errors, **you may need to restart your notebook kernel after you do the installation**.    
Make sure you're using Python 3. 
If you're running this notebook locally within a Conda environment, you may need to change `pip3` to `pip`.

In [ ]:
!pip3 install kfp --upgrade

Next, do some imports:

In [ ]:
import datetime

import kfp
import kfp.compiler as compiler
import kfp.dsl as dsl

from google.cloud import storage

Define a (very) simple example pipeline to run:

In [ ]:
@dsl.pipeline(
  name='Sequential',
  description='A pipeline with two sequential steps.'
)
def sequential_pipeline(filename='gs://ml-pipeline-playground/shakespeare1.txt'):
  """A pipeline with two sequential steps."""

  op1 = dsl.ContainerOp(
     name='getfilename',
     image='library/bash:4.4.23',
     command=['sh', '-c'],
     arguments=['echo "%s" > /tmp/results.txt' % filename],
     file_outputs={'newfile': '/tmp/results.txt'})
  op2 = dsl.ContainerOp(
     name='echo',
     image='library/bash:4.4.23',
     command=['sh', '-c'],
     arguments=['echo "%s"' % op1.outputs['newfile']]
     )

Next we'll create an instance of the Kubeflow Pipelines client. It will connect via the port-forward connection you set up.

In [ ]:
ts = int(datetime.datetime.utcnow().timestamp() * 100000)
client = kfp.Client('127.0.0.1:8089/pipeline')

Compile the pipeline, and create a Pipelines `Experiment`. (If you're running in an AI Platform notebook, the generated 'experiment' link will not work).

In [ ]:
compiler.Compiler().compile(sequential_pipeline, '/tmp/sequential.tar.gz')
exp = client.create_experiment(name='sequential')

Finally, run the pipeline.
If you're running this example in an AI Platform notebook, the generated 'run' link will not work. Instead, to view the Kubeflow and Kubeflow Pipelines dashboards, you can set up an additional port-forward from your local machine (a bit more info is [here](https://www.kubeflow.org/docs/other-guides/accessing-uis/)), or from the GCP [cloud shell](https://cloud.google.com/shell/docs/using-web-preview) using its 'Web Preview' feature.

In [ ]:
res =  client.run_pipeline(exp.id, 'sequential_' + str(ts), 
                           '/tmp/sequential.tar.gz',
                          )
print(res)

### Finding and using the ID of an existing pipeline to launch a run

In the scenario above, we ran a pipeline by uploading its compiled archive.
Sometimes, you might want to trigger a run of an existing pipeline, that has already been uploaded previously (e.g., via the `upload_pipeline` method, or via the web UI). 

For this, you need the ID of the pipeline.  You can get it via the `list_pipelines` method, filtering on the name of the pipeline that you want.

Here is an example wrapper method that takes as args the pipeline name and the client object, and returns the ID of that pipeline if it's in the list.

In [ ]:
def get_pipeline_id(name, client):
  pl_id = None
  page_size = 100
  page_token = ''
  while True:
    res = client.list_pipelines(page_size=page_size, page_token=page_token)
    pl_list = res.pipelines
    for pl in pl_list:
      if pl.name == name:
        pl_id = pl.id
        return pl_id
    page_token = res.next_page_token
    if not page_token:
      break
  return pl_id

We can use this helper method to trigger a pipeline run based on its ID. The pipeline with name "[Sample] Basic - Parallel execution" should exist as a sample in your Kubeflow installation, so let's use that as our pipeline.

In [ ]:
pipeline_name = '[Sample] Basic - Parallel execution'
pipeline_id = get_pipeline_id(name, client)
if pipeline_id:
  print("using pipeline ID: %s" pipeline_id)
  res = client.run_pipeline(exp.id, 'parallelexec' + str(ts),
                            # params={...},
                            pipeline_id=pipeline_id)
  else:
    print("Could not find Pipeline ID from name %s" % name)

Copyright 2019, Google, LLC.
Licensed under the Apache License, Version 2.0 (the "License");
you may not use this file except in compliance with the License.
You may obtain a copy of the License at

   http://www.apache.org/licenses/LICENSE-2.0

Unless required by applicable law or agreed to in writing, software
distributed under the License is distributed on an "AS IS" BASIS,
WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
See the License for the specific language governing permissions and
limitations under the License.